In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv

import os

In [19]:
def imshow(img, title=None):
    plt.imshow(img/(np.max(img)-np.min(img)))
    if title is not None:
        plt.title(title)
    plt.show()

In [34]:
def csv_to_npz(train_file, test_file, data_dir='data/'):
    train_data = pd.read_csv(train_file)
    test_data = pd.read_csv(test_file)

    x_train = train_data.values[:,1:].reshape((-1,28,28))
    y_train = np.asarray(train_data['label'])

    x_test = test_data.values[:,:].reshape((-1,28,28))
    
    np.savez(os.path.join(data_dir,'train'),x=x_train,y=y_train)
    np.savez(os.path.join(data_dir,'test'),x=x_test)

In [103]:
def transform(x, size=(256, 256)):
    x_new = np.zeros(size)
    h = np.random.randint(x.shape[0]/2,x.shape[0]*2,1)
    w = np.random.randint(x.shape[1]/2,x.shape[1]*2,1)
    
    x = cv.resize(x, (h,w))
    l = np.random.randint(0, size[1]-w,1)
    u = np.random.randint(0, size[0]-h,1)
    
    x_new[l[0]:l[0]+w[0], u[0]:u[0]+h[0]] = x
    return x_new

In [35]:
csv_to_npz('data/mnist/train.csv', 'data/mnist/test.csv')

In [36]:
train_data = np.load('data/train.npz')
test_data = np.load('data/test.npz')

In [108]:
x_train = train_data['x'].astype(np.float)
y_train = train_data['y']
x_test = test_data['x'].astype(np.float)

In [109]:
size = (256,256)
x_train_new = np.empty((x_train.shape[0], *size))
x_test_new = np.empty((x_test.shape[0], *size))

In [110]:
for i in range(x_train.shape[0]):
    x_train_new[i, :, :] = transform(x_train[i, :, :])

In [111]:
for i in range(x_test.shape[0]):
    x_test_new[i, :, :] = transform(x_test[i, :, :])

In [116]:
np.savez('data/train',x=x_train_new,y=y_train)
np.savez('data/test',x=x_test_new)